# Question Generation Examples

This notebook demonstrates how to use the LLM Test Generator for generating questions from Markdown snippets.

## Setup

Before running this notebook, ensure you have:
1. Created a `.env` file in the project root with your API keys (copy from `.env.example`)
2. Installed all dependencies: `pip install -r requirements.txt`
3. Set at least `OPENAI_API_KEY` in your `.env` file

In [12]:
SNIPPET = """## Метод K средних

Пожалуй, один из наиболее популярных методов кластеризации — это метод K-средних (K-means). Основная идея метода — итеративное повторение двух шагов:

1. распределение объектов выборки по кластерам;
2. пересчёт центров кластеров.

В начале работы алгоритма выбираются случайных центров в пространстве признаков. Каждый объект выборки относят к тому кластеру, к центру которого объект оказался ближе. Далее центры кластеров пересчитывают как среднее арифметическое векторов признаков всех вошедших в этот кластер объектов (то есть центр масс кластера). Как только мы обновили центры кластеров, объекты заново перераспределяются по ним, а затем можно снова уточнить положение центров. Процесс продолжается до тех пор, пока центры кластеров не перестанут меняться.

![kmeans](https://yastatic.net/s3/education-portal/media/kmeans_4a27aaf200_0c3fe72855.gif)

### Выбор начального приближения

Первый вопрос при выборе начального положения центров — как, выбирая центры из некоторого случайного распределения, не попасть в область пространства признаков, где нет точек выборки. Базовое решение — просто выбрать в качестве центров какие-то из объектов выборки.

Вторая потенциальная проблема — кучное размещение центров. В этом случае их начальное положение с большой вероятностью окажется далёким от итогового положения центров кластеров. Например, для таких изначальных положений центров

![19](https://yastatic.net/s3/education-portal/media/19_2_55eb1fbe5f_1bc4d07ce0.webp)

мы получим неправильную кластеризацию.

![19](https://yastatic.net/s3/education-portal/media/19_3_a081b9c158_06b66f9a43.webp)

Чтобы бороться с этим явлением, выгодно брать максимально удаленные друг от друга центры.

На практике работает следующая эвристика:

1. первый центр выбираем случайно из равномерного распределения на точках выборки;
2. каждый следующий центр выбираем из случайного распределения на объектах выборки, в котором вероятность выбрать объект пропорциональна квадрату расстояния от него до ближайшего к нему центра кластера.
Модификация K-means, использующая эту эвристику для выбора начальных приближений, называется K-means++.
"""

In [7]:
import sys
from pathlib import Path
from dotenv import load_dotenv
import json
from pprint import pprint

# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

# Load environment variables from project root
load_dotenv(project_root / ".env")

# Import the high-level API
from app.core.exam_builder import (
    generate_question,
    generate_exam_from_text,
    generate_exam_from_file,
    save_exam,
    ExamBuilder
)

print("✓ Setup complete!")

✓ Setup complete!


## Example 1: Generate a Single Question from Text Snippet

This is the simplest use case - generate one question from a small text snippet.

In [13]:
# Generate a single-choice question in Russian
question = generate_question(
    content=SNIPPET,
    question_type="single_choice",
    difficulty="medium",
    provider="openai",  # or "yandex" if you have Yandex credentials
    language="ru"
)

# Display the question
print("Generated Question:")
print(f"\nQuestion: {question['stem']}")
print(f"\nOptions:")
for i, option in enumerate(question['options']):
    marker = "✓" if i in question['correct'] else " "
    print(f"  [{marker}] {i+1}. {option}")
print(f"\nFull json:")
pprint(question)

Generated Question:

Question: Какой метод используется для выбора начальных центров в модификации K-means++, чтобы избежать кучного размещения центров?

Options:
  [ ] 1. Выбор центров из случайного распределения на объектах выборки
  [ ] 2. Выбор центров только среди крайних точек выборки
  [✓] 3. Выбор каждого следующего центра пропорционально квадрату расстояния до ближайшего центра
  [ ] 4. Выбор всех центров из одной области пространства признаков

Full json:
{'correct': [2],
 'options': ['Выбор центров из случайного распределения на объектах выборки',
             'Выбор центров только среди крайних точек выборки',
             'Выбор каждого следующего центра пропорционально квадрату '
             'расстояния до ближайшего центра',
             'Выбор всех центров из одной области пространства признаков'],
 'rubric': ['Правильно определяет метод выбора центров',
            'Объясняет, как работает эвристика выбора центров',
            'Понимает проблему кучного размещения це

## Example 2: Generate Multiple-Choice Question

Multiple-choice questions can have more than one correct answer.

In [21]:
# Generate a multiple-choice question
question = generate_question(
    content=SNIPPET,
    question_type="multiple_choice",
    difficulty="easy",
    provider="openai",
    language="ru"
)

print("Generated Multiple-Choice Question:")
print(f"\nQuestion: {question['stem']}")
print(f"\nOptions (select all that apply):")
for i, option in enumerate(question['options']):
    marker = "✓" if i in question['correct'] else " "
    print(f"  [{marker}] {i+1}. {option}")
print(f"\nCorrect answers: {len(question['correct'])} out of {len(question['options'])}")
print(f"\nFull json:")
pprint(question)

Generated Multiple-Choice Question:

Question: Какова основная идея метода K-средних?

Options (select all that apply):
  [✓] 1. Итеративное распределение объектов выборки по кластерам
  [ ] 2. Выбор центров кластеров, основываясь на случайном распределении
  [✓] 3. Пересчет центров кластеров как среднее арифметическое
  [ ] 4. Фиксирование центров кластеров на протяжении всего алгоритма

Correct answers: 2 out of 4

Full json:
{'correct': [0, 2],
 'options': ['Итеративное распределение объектов выборки по кластерам',
             'Выбор центров кластеров, основываясь на случайном распределении',
             'Пересчет центров кластеров как среднее арифметическое',
             'Фиксирование центров кластеров на протяжении всего алгоритма'],
 'rubric': ['Правильно определяет основные этапы метода K-средних',
            'Объясняет процесс пересчета центров кластеров',
            'Понимает значение итеративного подхода в алгоритме'],
 'stem': 'Какова основная идея метода K-средних?'}


## Example 3: Generate Open-Ended

In [23]:
# Generate a multiple-choice question
question = generate_question(
    content=SNIPPET,
    question_type="open_ended",
    difficulty="easy",
    provider="openai",
    language="ru"
)
print("Generated Open-Ended Question:")
print(f"\nQuestion: {question['stem']}")
print(f"\nFull json:")
pprint(question)

Generated Open-Ended Question:

Question: Как метод K-средних определяет центры кластеров и какую проблему помогает решить использование эвристики K-means++?

Full json:
{'reference_answer': 'Метод K-средних определяет центры кластеров путём '
                     'итеративного пересчёта средних значений векторов '
                     'признаков объектов, отнесённых к каждому кластеру. '
                     'Использование эвристики K-means++ помогает решить '
                     'проблему неэффективного выбора начальных центров, что '
                     'может привести к неправильной кластеризации, за счет '
                     'выбора центров, максимально удалённых друг от друга, что '
                     'повышает вероятность более точного распределения '
                     'объектов по кластерам.',
 'rubric': ['Определяет, как метод K-средних пересчитывает центры кластеров.',
            'Указывает на проблему, возникающую при выборе начальных центров '
            'без K-m

## Example 4: Generate Questions in English

The system supports both Russian (ru) and English (en) languages.

In [24]:
# Generate question in English
question = generate_question(
    content=SNIPPET,
    question_type="single_choice",
    difficulty="hard",
    provider="openai",
    language="en"
)

print("Generated English Question:")
print(f"\nQuestion: {question['stem']}")
print(f"\nOptions:")
for i, option in enumerate(question['options']):
    marker = "✓" if i in question['correct'] else " "
    print(f"  [{marker}] {i+1}. {option}")

Generated English Question:

Question: What is the primary heuristic used in the K-means++ algorithm for selecting initial cluster centers?

Options:
  [ ] 1. Selecting all centers randomly from the entire feature space
  [✓] 2. Choosing the first center randomly from the data points and subsequent centers based on the distance to the nearest center
  [ ] 3. Placing all centers in the densest area of the feature space
  [ ] 4. Choosing centers that are close together to minimize initial variance


## Example 5: Generate a Complete Exam from Text

Generate multiple questions at once from a larger text document.

In [42]:
# Generate an exam with 5 questions
exam = generate_exam_from_text(
    markdown_content=SNIPPET,
    total_questions=5,
    single_choice_ratio=0.2,      # 20% single-choice
    multiple_choice_ratio=0.4,    # 40% multiple-choice
    open_ended_ratio=0.4,         # 40% open-ended
    difficulty="mixed",           # mix of easy, medium, hard
    language="ru",
    seed=42                       # for reproducibility
)

# Display exam summary
print(f"Exam ID: {exam.exam_id}")
print(f"Total Questions: {len(exam.questions)}")
print(f"\nQuestions:")
print("=" * 80)

for i, q in enumerate(exam.questions, 1):
    print(f"\n{i}. [{q.type}] {q.stem}")
    try:
        for j, opt in enumerate(q.options):
            marker = "✓" if j in q.correct else " "
            print(f"   [{marker}] {chr(65+j)}. {opt}")
        print(f"   Difficulty: {q.meta.difficulty}")
    except TypeError:
        print(f"Reference answer:")
        print(q.reference_answer)
        print(f"Rubrics:")
        print(q.rubric)

Exam ID: ex-e536ea98
Total Questions: 5

Questions:

1. [multiple_choice] Какой метод используется для выбора начальных центров в модификации K-means++?
   [✓] A. Выбор случайного центра из точек выборки
   [ ] B. Выбор центров, находящихся близко друг к другу
   [✓] C. Выбор центров, максимально удалённых друг от друга
   [ ] D. Выбор центров только случайным образом без учета расстояний
   Difficulty: easy

2. [multiple_choice] Каковы основные этапы метода K-средних (K-means)?
   [✓] A. Выбор случайных центров кластеров
   [✓] B. Пересчет центров кластеров как среднее арифметическое
   [✓] C. Распределение объектов по кластерам
   [ ] D. Случайная сортировка объектов выборки
   Difficulty: easy

3. [open_ended] Объясните, как метод K-средних достигает своей цели кластеризации и какие факторы могут влиять на качество полученных кластеров.
Reference answer:
Метод K-средних достигает своей цели кластеризации, итеративно распределяя объекты по кластерам на основе близости к центрам класт

## Example 6: Generate Exam from Markdown File

Use the complete medical sample file to generate a full exam.

In [44]:
# Path to the sample medical file
sample_file = project_root / "examples" / "sample_data" / "sample_clustering.md"

# Generate exam from file
exam = generate_exam_from_file(
    file_path=str(sample_file),
    total_questions=10,
    single_choice_ratio=0.2,      # 20% single-choice
    multiple_choice_ratio=0.4,    # 40% multiple-choice
    open_ended_ratio=0.4,         # 40% open-ended
    difficulty="hard",
    language="ru"
)

print(f"Exam ID: {exam.exam_id}")
print(f"Total Questions: {len(exam.questions)}")
print(f"\nQuestions:")
print("=" * 80)

for i, q in enumerate(exam.questions, 1):
    print(f"\n{i}. [{q.type}] {q.stem}")
    try:
        for j, opt in enumerate(q.options):
            marker = "✓" if j in q.correct else " "
            print(f"   [{marker}] {chr(65+j)}. {opt}")
        print(f"   Difficulty: {q.meta.difficulty}")
    except TypeError:
        print(f"Reference answer:")
        print(q.reference_answer)
        print(f"Rubrics:")
        print(q.rubric)

Exam ID: ex-4590b735
Total Questions: 10

Questions:

1. [single_choice] Какая из следующих метрик качества кластеризации наиболее подходит для оценки плотности кластеров?
   [✓] A. Силуэтный коэффициент
   [ ] B. Метрика Дэвиса-Болдуина
   [ ] C. Внутрикластерное расстояние
   [ ] D. Индекс Калинского-Харабаса
   Difficulty: hard

2. [single_choice] Какое условие обычно используется для завершения работы алгоритма кластеризации?
   [✓] A. Получение заранее заданного количества кластеров
   [ ] B. Объединение кластеров на основе их средней плотности
   [ ] C. Слияние кластеров, пока не останется один кластер
   [ ] D. Увеличение расстояния между объединяемыми кластерами
   Difficulty: hard

3. [multiple_choice] Какое значение имеет выбор евклидовой метрики для метода K-средних в контексте пересчета центров кластеров?
   [✓] A. Евклидова метрика обеспечивает оптимальный выбор центров в среднем арифметическом.
   [✓] B. Евклидова метрика не влияет на шаг отнесения объектов к центрам.
   

## Example 7: Using Yandex GPT (if configured)

If you have Yandex Cloud credentials configured, you can use YandexGPT instead of OpenAI.

In [ ]:
# Check if Yandex credentials are configured
import os

if os.getenv("YANDEX_CLOUD_API_KEY"):
    snippet = """
   ### V-мера

**Гомогенность** и **полнота** кластеризации – это в некотором смысле аналоги точности и полноты классификации. Аналог F-меры для задачи кластеризации тоже есть, он называется V-мерой и связан с гомогенностью и полнотой той же формулой, что и F-мера с точностью и полнотой:
    """
    
    question = generate_question(
        content=snippet,
        question_type="single_choice",
        difficulty="medium",
        provider="yandex",  # Using Yandex instead of OpenAI
        language="ru"
    )
    
    print("Question generated with YandexGPT:")
    print(f"\n{question['stem']}\n")
    for i, opt in enumerate(question['options']):
        print(f"{chr(65+i)}. {opt}")
else:
    print("Yandex credentials not configured. Set YANDEX_CLOUD_API_KEY in .env to use YandexGPT.")

## Summary

This notebook demonstrated:

- ✅ Generating single questions from text snippets
- ✅ Different question types (single_choice, multiple_choice, open_ended)
- ✅ Multi-language support (Russian and English)
- ✅ Generating complete exams from longer texts
- ✅ Loading from Markdown files
- ✅ Using different AI providers (OpenAI, Yandex)

### Key Functions for The Team:

```python
# Quick single question generation
question = generate_question(content, question_type="single_choice", language="ru")

# Full exam from text
exam = generate_exam_from_text(markdown_content, total_questions=10, language="ru")

# Full exam from file
exam = generate_exam_from_file("path/to/file.md", total_questions=20, language="en")

# Save exam
path = save_exam(exam)

# Load exam
exam = load_exam("path/to/exam.json")
```

### For RAG Applications:

Use `generate_question()` on each retrieved snippet to create targeted questions based on relevant context.

### Troubleshooting:

- If you get "OPENAI_API_KEY not set": Check your `.env` file in the project root
- For Yandex: Set `YANDEX_CLOUD_API_KEY`, `YANDEX_CLOUD_API_KEY_IDENTIFIER`, and `YANDEX_FOLDER_ID`
- Questions in wrong language: Check the `language` parameter ("ru" or "en")